In [3]:
%load_ext autoreload

from pathlib import Path

In [13]:

%autoreload 2
from final_project.loader import get_df
import final_project.base.base as b


# file_name = 'Flights_2018_1.csv'
# exists = (l.res_dir / file_name).exists()
# print(exists)

# df = l.get_df(file='Flights_2018_2.csv')
df = get_df(year=2019)
# print(df)
# l.save_df("df_2018", df)
# for column in df.columns:
#     print(column)
# b.run()
# name = "Flights_2018_7.csv"
# parts = name.split("_")[1:3]
# year = parts[0]
# end = parts[1].split(".")[0]
# print(parts[0], end)

In [14]:
df = get_df(file="Flights_2018_1.csv")
print(df)
# file="df_2018.pkl"
# file="df_2018_1.pkl"
# file="Flights_2018_1.pkl"
# parts = file.split("_")
# print(f"{parts=}")

# back_end = parts[1:]
# if len(back_end) > 1:
#     year_part = back_end[0]
#     split = back_end[1].split(".")
#     num_part = "_" + split[0]
# else:
#     year_part = back_end[0].split(".")[0]
#     num_part = ""


# print(f"{year_part=}, {num_part=}")

        Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0       2018        1      1          23          2  2018-01-23   
1       2018        1      1          24          3  2018-01-24   
2       2018        1      1          25          4  2018-01-25   
3       2018        1      1          26          5  2018-01-26   
4       2018        1      1          27          6  2018-01-27   
...      ...      ...    ...         ...        ...         ...   
457683  2018        1      1          25          4  2018-01-25   
457684  2018        1      1          25          4  2018-01-25   
457685  2018        1      1          26          5  2018-01-26   
457686  2018        1      1          28          7  2018-01-28   
457687  2018        1      1          28          7  2018-01-28   

       Marketing_Airline_Network Operated_or_Branded_Code_Share_Partners  \
0                             DL                            DL_CODESHARE   
1                             DL           